In [35]:
import pandas as pd
import requests

In [36]:
!ls ~/Downloads/factbook-2017/fields | head -n10

2001.html
2002.html
2003.html
2004.html
2006.html
2007.html
2008.html
2010.html
2011.html
2012.html


In [37]:
from bs4 import BeautifulSoup

In [38]:
path = os.path.expanduser('~/Downloads/factbook-2017/docs/notesanddefs.html')
page = open(path).read()

In [39]:
page = BeautifulSoup(page)
print(page.prettify()[:1000])

<!DOCTYPE html>
<!--[if lt IE 7]> <html class="no-js lt-ie9 lt-ie8 lt-ie7" lang="en"> <![endif]-->
<!--[if IE 7]>    <html class="no-js lt-ie9 lt-ie8" lang="en"> <![endif]-->
<!--[if IE 8]>    <html class="no-js lt-ie9" lang="en"> <![endif]-->
<!--[if gt IE 8]><!-->
<!--<![endif]-->
<html class="no-js" lang="en">
 <!-- InstanceBegin template="/Templates/wfbext_template.dwt.cfm" codeOutsideHTMLIsLocked="false" -->
 <head>
  <meta charset="utf-8"/>
  <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
  <!-- InstanceBeginEditable name="doctitle" -->
  <title>
   The World Factbook
  </title>
  <!-- InstanceEndEditable -->
  <meta content="" name="description"/>
  <meta content="width=device-width" name="viewport"/>
  <link href="../css/fullscreen-external.css" rel="stylesheet" type="text/css"/>
  <script src="../js/modernizr-latest.js">
  </script>
  <!--developers version - switch to specific production http://modernizr.com/download/-->
  <script src="../js/jquery-1.8.3.min.

In [40]:
# Q-1: How many div tags are on the page? 
divs = page.select('div')
print(len(divs))

793


In [41]:
# Q-2: What kind of tag is the last tag to have the class of “cfclose”?
tags = page.select(".cfclose")
tags[-1]

<button aria-hidden="true" aria-label="Close" class="cfclose" data-dismiss="modal" type="button">×</button>

In [84]:
file_by_category = {}
cols = page.select("span.category")
for col in cols:
    cells = col.select('td')
    colname = cells[0].text
    links = cells[1].select('a')
    if len(links) > 0:
        fpath = links[0]['href']
        file_by_category[colname] = fpath.split('#')[0]

In [85]:
# Q-3: What is the path and filename for the file containing the data for “Internet users”? 
file_by_category['Internet users']

'../fields/2153.html'

In [117]:
file_by_category['Area']

'../fields/2147.html'

In [128]:
def extract_col(col_name):
    path = os.path.expanduser('~/Downloads/factbook-2017/docs/' + file_by_category[col_name])
    page = open(path).read()
    page = BeautifulSoup(page)

    rows = page.select('#fieldListing tr')
    rows

    result = {}
    
    for row in rows:
        if (row.has_attr('class') and 'fieldHeading' in row['class']):
            continue
        country_code = row['id']
        country_name = row.select('td > a')[0].text
        field_data = row.select('.fieldData')[0].text
        
        result[country_code] = field_data.strip()

    return result

    
list(extract_col("Area").items())[:2]

[('af', 'total: 652,230 sq km\nland: 652,230 sq km\nwater: 0 sq km'),
 ('ax', 'total: 123 sq km\nnote: includes a salt lake and wetlands')]

In [138]:
target_columns = """
Population
Area
Coastline
Climate
Net migration rate
Birth rate
Death rate
Infant mortality rate
Literacy
GDP (purchasing power parity)
Government type
Inflation rate (consumer prices)
Health expenditures
GDP - composition, by sector of origin
Land use
Internet users""".strip().split('\n')

result_dict = {}
for col_name in target_columns:
    result_dict[col_name] = extract_col(col_name)

In [170]:
df = pd.DataFrame(result_dict)

In [172]:
df[df['Infant mortality rate'] == 'total: NA\nmale: NA\nfemale: NA'] = 'NaN'

In [173]:
df['Infant mortality rate'] = df['Infant mortality rate'].str.split(' ', expand=True)[1]

In [175]:
df['Infant mortality rate'] = df['Infant mortality rate'].astype(float)

In [177]:
# Q-4: What is the average value for the column Infant mortality rate, to two significant digits? 
round(df['Infant mortality rate'].mean(), 2)

22.13